## 載入模型

In [212]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [213]:
from tensorflow.keras.models import load_model
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib  # 用於保存模型
import json
from sklearn.model_selection import train_test_split

In [214]:
# 載入模型與標準化器
base_model = load_model('Model_SAVE/1 mc AP RSSI FTM StdDev 2024-12-14 data train/DNN_best_modelDNN_3Layer_1AP_Best_combine_F_S_R_20241214.h5')

scaler = joblib.load('Model_SAVE/1 mc AP RSSI FTM StdDev 2024-12-14 data train/scaler_DNN_3Layer_1AP_Best_combine_F_S_R_20241214.pkl')

In [215]:
# 凍結所有層
for layer in base_model.layers[:-1]:  # 除了最後一層 (Output Layer)
    layer.trainable = False

# 確認哪些層可訓練
base_model.summary()

Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_310 (Dense)           (None, 64)                448       
                                                                 
 dense_311 (Dense)           (None, 128)               8320      
                                                                 
 dense_312 (Dense)           (None, 64)                8256      
                                                                 
 dense_313 (Dense)           (None, 49)                3185      
                                                                 
Total params: 20,209
Trainable params: 3,185
Non-trainable params: 17,024
_________________________________________________________________


In [216]:
base_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 加入新資料 & 資料前處理

In [217]:
selected_columns = ['Label',
                    'AP1_Distance (mm)',
                              'AP1_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

X_testing_selected_columns = ['AP1_Distance (mm)',
                              'AP1_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

label_column = 'Label'
target_column = 'Label'
#  'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi'
# 'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
# 'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',

In [ ]:
# 讀取測試資料
test_file_path = "timestamp_allignment_Balanced_2025_01_03_rtt_logs.csv"  # 測試資料的檔案名稱
date_test = "2025_01_03"
modelname = "DNN 1 mcAPs BEST"
test_data = pd.read_csv(test_file_path, usecols=selected_columns)
test_data

,Label,AP1_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm)
0,1-1,3960.0,-62.0,-72.0,-64.0,-47.0,892.0
1,1-1,4077.0,-62.0,-72.0,-64.0,-47.0,920.0
2,1-1,4038.0,-57.0,-72.0,-60.0,-45.0,1116.0
3,1-1,3960.0,-62.0,-72.0,-63.0,-47.0,979.0
4,1-1,3960.0,-62.0,-72.0,-63.0,-48.0,924.0
...,...,...,...,...,...,...,...
21359,9-11,1944.0,-48.0,-67.0,-55.0,-63.0,699.0
21360,9-11,2124.0,-56.0,-66.0,-56.0,-63.0,165.0
21361,9-11,1968.0,-55.0,-66.0,-56.0,-61.0,334.0
21362,9-11,2163.0,-56.0,-66.0,-56.0,-63.0,139.0


In [219]:
# 資料前處理 (一): 刪除前後n筆資料
n = 20
# 確保依據Label排序
test_data = test_data.sort_values(by=label_column).reset_index(drop=True)

# 建立一個空的 DataFrame 用於存放處理後的資料
test_processed_data = pd.DataFrame(columns=test_data.columns)

# 針對每個Label群組進行處理
for label, group in test_data.groupby(label_column):
    # 刪除前n筆和後n筆資料
    if len(group) > 2 * n:  # 確保群組資料足夠
        group = group.iloc[n:-n]
    else:
        group = pd.DataFrame()  # 若資料不足，刪除整個群組
    # 將處理後的群組資料加入
    test_processed_data = pd.concat([test_processed_data, group], ignore_index=True)

# test_processed_data
# Calculate the number of rows with NaN values
nan_rows = test_processed_data.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_processed_data[test_processed_data.isnull().any(axis=1)]

# # 印出這些列
# print("Rows with NaN values:")
# print(rows_with_nan)
test_data_imputed = test_processed_data.groupby(label_column).apply(
    lambda group: group.fillna(group.mean())
).reset_index()

# Calculate the number of rows with NaN values
nan_rows = test_data_imputed.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_data_imputed[test_data_imputed.isnull().any(axis=1)]

test_data_imputed

Number of rows with NaN values: 1468
Number of rows with NaN values: 0


,Label,level_1,AP1_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm)
0,1-1,0,3921.0,-62.0,-72.0,-62.0,-47.0,1123.0
1,1-1,1,3803.0,-62.0,-72.0,-64.0,-48.0,1186.0
2,1-1,2,4343.0,-52.0,-72.0,-64.0,-43.0,1459.0
3,1-1,3,3843.0,-62.0,-72.0,-64.0,-48.0,1155.0
4,1-1,4,3843.0,-62.0,-72.0,-64.0,-47.0,1146.0
...,...,...,...,...,...,...,...,...
19399,9-11,19399,2202.0,-56.0,-67.0,-53.0,-64.0,183.0
19400,9-11,19400,2046.0,-48.0,-66.0,-57.0,-58.0,373.0
19401,9-11,19401,2124.0,-55.0,-67.0,-54.0,-64.0,289.0
19402,9-11,19402,2163.0,-55.0,-66.0,-56.0,-63.0,223.0


In [220]:
reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1

In [221]:
# 建立 Label 映射
y_test = test_data_imputed[target_column]
y_test_numeric = y_test.map(reverse_label_mapping)

print("Final reverse_label_mapping in DNN:", reverse_label_mapping)
print("y_numeric unique values in DNN:", y_test_numeric.unique())

# 把label部分拿掉
X_test = test_data_imputed.drop(columns=['level_1','Label'])
# 確保測試資料的特徵與訓練資料的特徵一致
X_test = X_test[X_testing_selected_columns]  # 選取相同的特徵

X_test

Final reverse_label_mapping in DNN: {'1-1': 10, '1-2': 9, '1-3': 8, '1-4': 7, '1-5': 6, '1-6': 5, '1-7': 4, '1-8': 3, '1-9': 2, '1-10': 1, '1-11': 0, '2-1': 11, '2-11': 29, '3-1': 12, '3-11': 28, '4-1': 13, '4-11': 27, '5-1': 14, '5-11': 26, '6-1': 15, '6-2': 16, '6-3': 17, '6-4': 18, '6-5': 19, '6-6': 20, '6-7': 21, '6-8': 22, '6-9': 23, '6-10': 24, '6-11': 25, '7-1': 48, '7-11': 30, '8-1': 47, '8-11': 31, '9-1': 46, '9-11': 32, '10-1': 45, '10-11': 33, '11-1': 44, '11-2': 43, '11-3': 42, '11-4': 41, '11-5': 40, '11-6': 39, '11-7': 38, '11-8': 37, '11-9': 36, '11-10': 35, '11-11': 34}
y_numeric unique values in DNN: [10  1  0  9  8  7  6  5  4  3  2 45 33 44 35 34 43 42 41 40 39 38 37 36
 11 29 12 28 13 27 14 26 15 24 25 16 17 18 19 20 21 22 23 48 30 47 31 46
 32]


,AP1_Distance (mm),AP1_StdDev (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi
0,3921.0,1123.0,-62.0,-72.0,-62.0,-47.0
1,3803.0,1186.0,-62.0,-72.0,-64.0,-48.0
2,4343.0,1459.0,-52.0,-72.0,-64.0,-43.0
3,3843.0,1155.0,-62.0,-72.0,-64.0,-48.0
4,3843.0,1146.0,-62.0,-72.0,-64.0,-47.0
...,...,...,...,...,...,...
19399,2202.0,183.0,-56.0,-67.0,-53.0,-64.0
19400,2046.0,373.0,-48.0,-66.0,-57.0,-58.0
19401,2124.0,289.0,-55.0,-67.0,-54.0,-64.0
19402,2163.0,223.0,-55.0,-66.0,-56.0,-63.0


In [222]:
# 使用之前訓練時的標準化器 (scaler) 來標準化測試數據
X_test_scaled = scaler.transform(X_test)

print(len(X_test_scaled))

19404


### 重新訓練model

In [223]:
# 取出 10% 資料作為訓練資料，剩下的 90% 作為測試資料
X_train_small, X_unused, y_train_small, y_unused = train_test_split(
    X_test_scaled, y_test_numeric, test_size=0.9, random_state=42, stratify=y_test_numeric
)

In [224]:
import time
# 記錄開始時間
start_time = time.time()

In [225]:
base_model.fit(X_train_small, y_train_small, epochs=20, batch_size=32)

Epoch 1/20
61/61 [==============================] - 0s 800us/step - loss: 18.7764 - accuracy: 0.2062
Epoch 2/20
61/61 [==============================] - 0s 810us/step - loss: 13.8265 - accuracy: 0.2536
Epoch 3/20
61/61 [==============================] - 0s 585us/step - loss: 10.4531 - accuracy: 0.3227
Epoch 4/20
61/61 [==============================] - 0s 547us/step - loss: 7.9854 - accuracy: 0.3907
Epoch 5/20
61/61 [==============================] - 0s 604us/step - loss: 6.0212 - accuracy: 0.4531
Epoch 6/20
61/61 [==============================] - 0s 608us/step - loss: 4.4665 - accuracy: 0.5222
Epoch 7/20
61/61 [==============================] - 0s 687us/step - loss: 3.3452 - accuracy: 0.6031
Epoch 8/20
61/61 [==============================] - 0s 457us/step - loss: 2.5814 - accuracy: 0.6711
Epoch 9/20
61/61 [==============================] - 0s 337us/step - loss: 2.0569 - accuracy: 0.7191
Epoch 10/20
61/61 [==============================] - 0s 320us/step - loss: 1.6790 - accuracy: 0.7

In [226]:
# 記錄結束時間
end_time = time.time()

# 計算訓練時間（秒）
training_time = end_time - start_time
print(f"訓練時間：{training_time:.2f} 秒")

訓練時間：0.73 秒


In [227]:
# 使用剩餘的 90% 測試資料進行模型評估
loss, accuracy = base_model.evaluate(X_unused, y_unused)
print(f"Evaluation on 90% unused data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

546/546 [==============================] - 0s 300us/step - loss: 0.7015 - accuracy: 0.8954
Evaluation on 90% unused data - Loss: 0.7015, Accuracy: 0.8954


In [228]:
# ### 預測
# base_model.evaluate(X_test_scaled, y_test_numeric)  # 確保模型在測試模式

In [229]:
# 預測測試資料
y_test_pred_numeric = base_model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_test_pred_numeric, axis=1)

# 轉換為原本的 Label
y_test_pred_labels = [label_mapping[str(num + 1)] for num in y_pred_classes]  # 補回 +1
y_test_pred_labels

607/607 [==============================] - 0s 350us/step


['1-1',
 '1-1',
 '1-2',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 '1-2',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-2',
 '2-1',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-2',
 '1-1',
 '1-1',
 '1-2',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-2',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-2',
 '1-1',
 '1-1',
 '2-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',


In [230]:
# 讀取測試資料的實際 Label
y_test_actual = test_data_imputed[target_column]
test_data_imputed

,Label,level_1,AP1_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm)
0,1-1,0,3921.0,-62.0,-72.0,-62.0,-47.0,1123.0
1,1-1,1,3803.0,-62.0,-72.0,-64.0,-48.0,1186.0
2,1-1,2,4343.0,-52.0,-72.0,-64.0,-43.0,1459.0
3,1-1,3,3843.0,-62.0,-72.0,-64.0,-48.0,1155.0
4,1-1,4,3843.0,-62.0,-72.0,-64.0,-47.0,1146.0
...,...,...,...,...,...,...,...,...
19399,9-11,19399,2202.0,-56.0,-67.0,-53.0,-64.0,183.0
19400,9-11,19400,2046.0,-48.0,-66.0,-57.0,-58.0,373.0
19401,9-11,19401,2124.0,-55.0,-67.0,-54.0,-64.0,289.0
19402,9-11,19402,2163.0,-55.0,-66.0,-56.0,-63.0,223.0


In [231]:
# 取得預測與實際座標
y_test_pred_coordinates = np.array([label_to_coordinates[label] for label in y_test_pred_labels])
y_test_actual_coordinates = np.array([label_to_coordinates[label] for label in y_test_actual])

# 計算 MDE (Mean Distance Error)
distances = np.linalg.norm(y_test_pred_coordinates - y_test_actual_coordinates, axis=1)
mean_mde = np.mean(distances)

# 記錄每個 RP 的 MDE
mde_report_test = {}
for true_label, distance in zip(y_test_actual, distances):
    if true_label not in mde_report_test:
        mde_report_test[true_label] = []
    mde_report_test[true_label].append(distance)

# 計算測試資料的 MDE 平均值
mde_report_test_avg = {label: {"mde": np.mean(dists), "count": len(dists)} for label, dists in mde_report_test.items()}

# 儲存 MDE 結果到 JSON 檔案
test_file_path = f"{modelname}_{date_test}.json"
with open(test_file_path, "w") as f:
    json.dump(mde_report_test_avg, f, indent=4)

print(f"Test Data MDE report saved to: {test_file_path}")
print(f"\nTest Data Mean MDE: {mean_mde:.4f} meters")


Test Data MDE report saved to: DNN 1 mcAPs BEST_2024_12_27.json

Test Data Mean MDE: 0.2246 meters
